# Don't Overfit! Baseline

# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', None)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import time

import warnings
warnings.filterwarnings('ignore')

# Read the training and testing data


In [ ]:
train_df = pd.read_csv("https://github.com/mohammad2012191/Don-t-Overfit-UofK/raw/main/Train.csv")
test_df = pd.read_csv("https://github.com/mohammad2012191/Don-t-Overfit-UofK/raw/main/Test.csv")

In [ ]:
ID = test_df['ID']
train_df.drop('ID',inplace=True,axis=1)
test_df.drop('ID',inplace=True,axis=1)

# Exploratory data analysis

# Features:

In [ ]:
Definitions = pd.read_csv('https://github.com/mohammad2012191/Don-t-Overfit-UofK/raw/main/VariableDefinitions.csv')
Definitions.drop('For columns(Q1 - Q28), Value 1 - Yes, 2 - No, 3-Don’t Know 4 - refused to answer',axis=1)

,ID,Person unique/random identification
0,country,The country
1,region,"Region of a given country,.i.e., South Asia, E..."
2,population,The population of the given country
3,age,Individual age
4,Q1,Has ATM/debit card
5,Q2,Is the ATM connect to an account with your name?
6,Q3,Purchased with the ATM
7,Q4,Use of mobile or internet to make payments or ...
8,Q5,Mobile/internet to check account balance
9,Q6,Has a credit card


In [ ]:
train_df.shape

(104867, 35)

In [ ]:
test_df.shape

(44943, 34)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104867 entries, 0 to 104866
Data columns (total 35 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      104867 non-null  object 
 1   region       104188 non-null  object 
 2   population   104867 non-null  float64
 3   age          104580 non-null  float64
 4   Q1           104867 non-null  int64  
 5   Q2           47721 non-null   float64
 6   Q3           45214 non-null   float64
 7   Q4           57445 non-null   float64
 8   Q5           57445 non-null   float64
 9   Q6           104867 non-null  int64  
 10  Q7           20641 non-null   float64
 11  Q8           59029 non-null   float64
 12  Q9           59029 non-null   float64
 13  Q10a         104867 non-null  int64  
 14  Q10b         104867 non-null  int64  
 15  Q11          104867 non-null  int64  
 16  Q12          104867 non-null  int64  
 17  Q13a         104867 non-null  int64  
 18  Q14          104867 non-

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44943 entries, 0 to 44942
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      44943 non-null  object 
 1   region       44642 non-null  object 
 2   population   44943 non-null  float64
 3   age          44819 non-null  float64
 4   Q1           44943 non-null  int64  
 5   Q2           20416 non-null  float64
 6   Q3           19348 non-null  float64
 7   Q4           24559 non-null  float64
 8   Q5           24559 non-null  float64
 9   Q6           44943 non-null  int64  
 10  Q7           8874 non-null   float64
 11  Q8           25243 non-null  float64
 12  Q9           25243 non-null  float64
 13  Q10a         44943 non-null  int64  
 14  Q10b         44943 non-null  int64  
 15  Q11          44943 non-null  int64  
 16  Q12          44943 non-null  int64  
 17  Q13a         44943 non-null  int64  
 18  Q14          44943 non-null  int64  
 19  Q15 

In [ ]:
train_df.head()

,country,region,population,age,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10a,Q10b,Q11,Q12,Q13a,Q14,Q15,Q16,Q17a,Q17b,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,owns_mobile,Q28,target
0,Morocco,Middle East & North Africa (excluding high inc...,25550170.000,66.000,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,2,2,2,2,2,2,1,2,2,1,2.000,1.000,1,2,NaN,1,2,2.000,2.000,1.000,1.000,0
1,Lebanon,Middle East & North Africa (excluding high inc...,4591023.000,44.000,2,NaN,NaN,2.000,2.000,2,NaN,1.000,1.000,2,2,2,2,1,2,2,2,1,2,2.000,2.000,1,2,NaN,2,2,2.000,1.000,1.000,1.000,1
2,Morocco,Middle East & North Africa (excluding high inc...,25550170.000,29.000,1,1.000,1.000,2.000,2.000,2,NaN,1.000,1.000,2,2,2,2,2,2,2,2,2,2,2.000,2.000,1,1,2.000,2,2,2.000,NaN,1.000,1.000,0
3,Canada,High income: OECD,30491168.000,44.000,1,1.000,1.000,1.000,1.000,2,NaN,1.000,1.000,1,1,2,1,1,2,2,2,1,2,NaN,NaN,1,1,2.000,1,2,NaN,NaN,1.000,NaN,1
4,Mexico,Latin America & Caribbean (excluding high income),93002320.000,36.000,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,1,2,2,2,2,2,2,2,2,2,2.000,2.000,1,2,NaN,1,2,2.000,2.000,1.000,1.000,1


In [ ]:
test_df.head()

,country,region,population,age,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10a,Q10b,Q11,Q12,Q13a,Q14,Q15,Q16,Q17a,Q17b,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,owns_mobile,Q28
0,Croatia,Europe & Central Asia (excluding high income),3557444.500,62.000,2,NaN,NaN,2.000,2.000,1,1.000,1.000,1.000,2,3,2,1,1,2,2,2,2,2,2.000,2.000,2,1,1.000,2,2,2.000,NaN,1.000,1.000
1,Kazakhstan,Europe & Central Asia (excluding high income),12920168.000,32.000,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,2,2,2,2,2,3,2,2,2,2,2.000,2.000,2,2,NaN,2,2,2.000,2.000,1.000,2.000
2,Costa Rica,Latin America & Caribbean (excluding high income),3791520.000,73.000,1,1.000,1.000,2.000,2.000,2,NaN,2.000,2.000,2,2,2,2,2,2,2,2,2,2,2.000,2.000,2,2,NaN,2,1,2.000,1.000,1.000,1.000
3,Japan,High income: OECD,110566824.000,67.000,1,1.000,1.000,1.000,2.000,1,1.000,1.000,1.000,1,1,2,1,1,2,2,2,2,2,NaN,NaN,1,1,2.000,1,1,NaN,NaN,1.000,NaN
4,Morocco,Middle East & North Africa (excluding high inc...,25550170.000,32.000,1,1.000,1.000,2.000,2.000,2,NaN,2.000,1.000,2,2,2,2,2,2,2,2,2,1,2.000,1.000,2,2,NaN,2,2,2.000,2.000,1.000,1.000


In [ ]:
train_df.describe()

,population,age,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10a,Q10b,Q11,Q12,Q13a,Q14,Q15,Q16,Q17a,Q17b,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,owns_mobile,Q28,target
count,104867.000,104580.000,104867.000,47721.000,45214.000,57445.000,57445.000,104867.000,20641.000,59029.000,59029.000,104867.000,104867.000,104867.000,104867.000,104867.000,104867.000,104867.000,104867.000,104867.000,104867.000,80694.000,80694.000,104867.000,104867.000,37580.000,104867.000,104867.000,80694.000,58429.000,104866.000,73862.000,104867.000
mean,67288272.366,41.821,1.556,1.060,1.298,1.619,1.556,1.819,1.154,1.220,1.203,1.790,1.809,1.868,1.792,1.765,1.874,1.906,1.938,1.885,1.780,1.807,1.778,1.432,1.650,1.746,1.862,1.881,1.856,1.846,1.176,1.126,0.582
std,209552863.799,17.824,0.523,0.274,0.466,0.496,0.509,0.429,0.373,0.439,0.427,0.431,0.421,0.377,0.442,0.459,0.357,0.320,0.269,0.349,0.437,0.419,0.441,0.516,0.499,0.452,0.375,0.357,0.376,0.393,0.394,0.343,0.493
min,373991.156,15.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000
25%,4335264.000,27.000,1.000,1.000,1.000,1.000,1.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,1.000,1.000,0.000
50%,9386639.000,39.000,2.000,1.000,1.000,2.000,2.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,1.000,1.000
75%,30519122.000,55.000,2.000,1.000,2.000,2.000,2.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,1.000,1.000
max,1134633728.000,99.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,1.000


In [ ]:
test_df.describe()

,population,age,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10a,Q10b,Q11,Q12,Q13a,Q14,Q15,Q16,Q17a,Q17b,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,owns_mobile,Q28
count,44943.000,44819.000,44943.000,20416.000,19348.000,24559.000,24559.000,44943.000,8874.000,25243.000,25243.000,44943.000,44943.000,44943.000,44943.000,44943.000,44943.000,44943.000,44943.000,44943.000,44943.000,34512.000,34512.000,44943.000,44943.000,16000.000,44943.000,44943.000,34512.000,25106.000,44942.000,31510.000
mean,69330044.695,41.755,1.558,1.060,1.291,1.613,1.549,1.818,1.165,1.215,1.199,1.790,1.806,1.868,1.794,1.761,1.874,1.906,1.937,1.882,1.780,1.807,1.778,1.434,1.652,1.745,1.860,1.878,1.855,1.848,1.180,1.128
std,213840648.795,17.911,0.525,0.276,0.460,0.497,0.512,0.431,0.382,0.437,0.426,0.434,0.423,0.374,0.440,0.458,0.355,0.320,0.268,0.348,0.433,0.417,0.440,0.515,0.496,0.452,0.375,0.357,0.377,0.391,0.399,0.349
min,373991.156,15.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
25%,4300062.000,27.000,1.000,1.000,1.000,1.000,1.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,1.000,1.000
50%,9386639.000,39.000,2.000,1.000,1.000,2.000,2.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,1.000
75%,30519122.000,55.000,2.000,1.000,2.000,2.000,2.000,2.000,1.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,1.000,1.000
max,1134633728.000,99.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000


# Plots


In [ ]:
train_df.head()

,country,region,population,age,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10a,Q10b,Q11,Q12,Q13a,Q14,Q15,Q16,Q17a,Q17b,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,owns_mobile,Q28,target
0,Morocco,Middle East & North Africa (excluding high inc...,25550170.000,66.000,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,2,2,2,2,2,2,1,2,2,1,2.000,1.000,1,2,NaN,1,2,2.000,2.000,1.000,1.000,0
1,Lebanon,Middle East & North Africa (excluding high inc...,4591023.000,44.000,2,NaN,NaN,2.000,2.000,2,NaN,1.000,1.000,2,2,2,2,1,2,2,2,1,2,2.000,2.000,1,2,NaN,2,2,2.000,1.000,1.000,1.000,1
2,Morocco,Middle East & North Africa (excluding high inc...,25550170.000,29.000,1,1.000,1.000,2.000,2.000,2,NaN,1.000,1.000,2,2,2,2,2,2,2,2,2,2,2.000,2.000,1,1,2.000,2,2,2.000,NaN,1.000,1.000,0
3,Canada,High income: OECD,30491168.000,44.000,1,1.000,1.000,1.000,1.000,2,NaN,1.000,1.000,1,1,2,1,1,2,2,2,1,2,NaN,NaN,1,1,2.000,1,2,NaN,NaN,1.000,NaN,1
4,Mexico,Latin America & Caribbean (excluding high income),93002320.000,36.000,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,1,2,2,2,2,2,2,2,2,2,2.000,2.000,1,2,NaN,1,2,2.000,2.000,1.000,1.000,1


In [ ]:
for c in train_df.columns:
    col_type = train_df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        train_df[c] = train_df[c].astype('category')

for c in test_df.columns:
    col_type = test_df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        test_df[c] = test_df[c].astype('category')

In [ ]:
!pip3 install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.metrics import classification_report
import optuna
from optuna.samplers import TPESampler
from optuna import Trial
import gc
import optuna
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
X = train_df.drop('target', axis=1)
y = train_df['target'].copy()

In [ ]:
def objective(trial: Trial):
    gc.collect()
    models=[]
    validScore=0
   
    model,log = fitLGB(trial, X, y)
    
    models.append(model)
    gc.collect()
    validScore+=log
    validScore/=len(models)
    
    return validScore

In [ ]:
def fitLGB(trial,X, y):
    params={
        'seed': 42,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100)
      }

    stkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    model = LGBMClassifier(**params)
    
    res=[]
    for i, (tdx, vdx) in enumerate(stkfold.split(X, y)):
        X_train, X_valid, y_train, y_valid = X.iloc[tdx], X.iloc[vdx], y[tdx], y[vdx]
        model.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_valid, y_valid)],
                 early_stopping_rounds=30, verbose=False)
        preds = model.predict_proba(X_valid)
        res.append(roc_auc_score(y_valid, preds[:,1]))
    
    err = np.mean(res)
    
    return model, err

In [ ]:
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, timeout=60*60*2)

[I 2023-03-10 22:40:10,404] A new study created in memory with name: no-name-24e3143d-95c4-46fe-a866-ecfab3a1290b
[I 2023-03-10 22:40:17,499] Trial 0 finished with value: 0.7944805625558565 and parameters: {'lambda_l1': 7.17288455903682e-05, 'lambda_l2': 0.0006682314204843411, 'num_leaves': 387, 'feature_fraction': 0.11265496812059146, 'bagging_fraction': 0.4094395572921341, 'bagging_freq': 12, 'min_child_samples': 19}. Best is trial 0 with value: 0.7944805625558565.
[I 2023-03-10 22:40:34,228] Trial 1 finished with value: 0.8002215796423275 and parameters: {'lambda_l1': 0.033761571653290544, 'lambda_l2': 1.283415489440248e-08, 'num_leaves': 446, 'feature_fraction': 0.6642481194576022, 'bagging_fraction': 0.7137891985980178, 'bagging_freq': 15, 'min_child_samples': 94}. Best is trial 1 with value: 0.8002215796423275.
[I 2023-03-10 22:40:50,521] Trial 2 finished with value: 0.79992515726214 and parameters: {'lambda_l1': 0.004873808470827652, 'lambda_l2': 1.0493259620125927e-05, 'num_lea

In [ ]:
best_params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "seed": 42
} 
best_params.update(study.best_params)
best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'seed': 42,
 'lambda_l1': 4.305078156378706,
 'lambda_l2': 0.03147091623961639,
 'num_leaves': 99,
 'feature_fraction': 0.6320722568346393,
 'bagging_fraction': 0.6394141167922078,
 'bagging_freq': 0,
 'min_child_samples': 83}

In [ ]:
#Train The model on the full dataset
lg = LGBMClassifier(**best_params, verbose_eval = False)
lg.fit(X,y)

#Make Predictions
preds = lg.predict_proba(test_df)[:,1]

In [ ]:
#Make the Submission file
submission = pd.DataFrame({"Id": ID ,"target": preds})
submission.to_csv('lgbm_optuna_finetunning_should_be_completed.csv',index=False)